## outline of solution
* identify the problem: build a machin learning model to predict if what type of browser users are on when accessing Bitly links. It is a multi-classes classification problem.

* feature engineering
    * extract target from a (browser user agent)
    * extract features as input for machine learning model

* model evaluation 

In [1]:
import psycopg2 as pg
import sys
import pickle

import json
import pprint

import numpy as np
import pandas as pd
from pandas import DataFrame, Series
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

import seaborn as sns
sns.set(style="whitegrid", color_codes=True)

from user_agents import parse

In [2]:
# read json data
json_holder=[]
with open("decodesaa_test","r") as f:
    for line in f:
        try:
            json_line=json.loads(line)
            json_holder.append(json_line)
        except:
            pass
print(len(json_holder))

100


In [42]:
pprint.pprint(json_holder[:2])

[{'a': 'Mozilla/5.0 (Linux; Android 4.4.2; GT-I9507 Build/KOT49H) '
       'AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/30.0.0.0 '
       'Mobile Safari/537.36 [FB_IAB/FB4A;FBAV/28.0.0.20.16;]',
  'c': 'AU',
  'cy': 'Perth',
  'g': '1H8wL5d',
  'h': '1H8wL5c',
  'nk': 1,
  't': 1427288399,
  'tz': 'Australia/West',
  'u': 'http://www.airasia.com/au/en/promotion/rr0550905.page?utm_source=facebook\\u0026utm_medium=organic\\u0026utm_campaign=aa_au_se\\u0026utm_term=SE_Mar15_AU'},
 {'a': 'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; '
       'Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET4.0C; .NET4.0E; 360SE)',
  'c': 'US',
  'cy': 'Mountain View',
  'g': '1lj67KQ',
  'h': '1xupVE6',
  'mc': 807,
  'nk': 0,
  't': 1427288399,
  'tz': 'America/Los_Angeles',
  'u': 'https://cdn.adf.ly/js/display.js'}]


## feature enginnering 

In [44]:
from pandas.io.json import json_normalize

In [45]:
json_normalize?

In [47]:
data = [{'state': 'Florida',
        'shortname': 'FL',
          'info': {
              'governor': 'Rick Scott'
        },
        'counties': [{'name': 'Dade', 'population': 12345},
                     {'name': 'Broward', 'population': 40000},
                     {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
        'shortname': 'OH',
          'info': {
              'governor': 'John Kasich'
         },
         'counties': [{'name': 'Summit', 'population': 1234},
                     {'name': 'Cuyahoga', 'population': 1337}]}]
pprint.pprint(data)

[{'counties': [{'name': 'Dade', 'population': 12345},
               {'name': 'Broward', 'population': 40000},
               {'name': 'Palm Beach', 'population': 60000}],
  'info': {'governor': 'Rick Scott'},
  'shortname': 'FL',
  'state': 'Florida'},
 {'counties': [{'name': 'Summit', 'population': 1234},
               {'name': 'Cuyahoga', 'population': 1337}],
  'info': {'governor': 'John Kasich'},
  'shortname': 'OH',
  'state': 'Ohio'}]


In [55]:
json_normalize(data, record_path=['counties'], 
               meta=['state', 'shortname', ['info', 'governor']])

,name,population,info.governor,shortname,state
0,Dade,12345,Rick Scott,FL,Florida
1,Broward,40000,Rick Scott,FL,Florida
2,Palm Beach,60000,Rick Scott,FL,Florida
3,Summit,1234,John Kasich,OH,Ohio
4,Cuyahoga,1337,John Kasich,OH,Ohio


In [49]:
json_normalize?

In [4]:
# extract label and features from json
# label is browser
# features include t, c, nk, kw, ckw, cy, tz
df = []

for item in json_holder[:100000]:
    try: # try to extract broswer from user agent string
        browser = parse(item['a']).browser.family
    except:
        broswer = None
        
    # extract features
    user_hash = item.get('h', None)
    global_hash = item.get('g', None)
    url = item.get('u', None)
    time = item.get('t', None)
    country = item.get('c', None)
    repeat_client = item.get('nk', None)
    keyword_alias = item.get('kw', None) 
    custom_keyword = item.get('ckw', None)
    city = item.get('cy', None)
    timezone = item.get('tz', None)
    
    df.append([browser, user_hash, global_hash, url, time, country, repeat_client, 
               keyword_alias, custom_keyword, city, timezone])
    df1 = pd.DataFrame(df, columns=['browser', 'user_hash', 'global_hash', 'url', 'time', 
                                    'country', 'repeat_client', 'keyword_alias', 
                                    'custom_keyword', 'city', 'timezone'])

In [5]:
del df

In [6]:
df1.head()

,browser,user_hash,global_hash,url,time,country,repeat_client,keyword_alias,custom_keyword,city,timezone
0,Facebook,1H8wL5c,1H8wL5d,http://www.airasia.com/au/en/promotion/rr05509...,1427288399,AU,1,None,None,Perth,Australia/West
1,IE,1xupVE6,1lj67KQ,https://cdn.adf.ly/js/display.js,1427288399,US,0,None,None,Mountain View,America/Los_Angeles
2,IE,1xupVE6,1lj67KQ,https://cdn.adf.ly/js/display.js,1427288399,US,0,None,None,Buffalo,America/New_York
3,Facebook,1z7y10j,1z7y10k,https://predikta.com/Account/Register,1427288399,GB,1,None,None,Leicester,Europe/London
4,Mobile Safari,1pv5AIm,1pv5AIn,http://www.hasbro.com/en_US/brands/telepods/tr...,1427288399,CN,0,RTLPHT0814,None,Shanghai,Asia/Shanghai


In [7]:
# convert categorical label into dummy values
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df1['label'] = le.fit_transform(df1.browser)

In [8]:
df1.head()

,browser,user_hash,global_hash,url,time,country,repeat_client,keyword_alias,custom_keyword,city,timezone,label
0,Facebook,1H8wL5c,1H8wL5d,http://www.airasia.com/au/en/promotion/rr05509...,1427288399,AU,1,None,None,Perth,Australia/West,3
1,IE,1xupVE6,1lj67KQ,https://cdn.adf.ly/js/display.js,1427288399,US,0,None,None,Mountain View,America/Los_Angeles,5
2,IE,1xupVE6,1lj67KQ,https://cdn.adf.ly/js/display.js,1427288399,US,0,None,None,Buffalo,America/New_York,5
3,Facebook,1z7y10j,1z7y10k,https://predikta.com/Account/Register,1427288399,GB,1,None,None,Leicester,Europe/London,3
4,Mobile Safari,1pv5AIm,1pv5AIn,http://www.hasbro.com/en_US/brands/telepods/tr...,1427288399,CN,0,RTLPHT0814,None,Shanghai,Asia/Shanghai,7


In [9]:
df1.url.apply(lambda url: url.startswith('https'))

0     False
1      True
2      True
3      True
4     False
5      True
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19     True
20     True
21    False
22    False
23    False
24    False
25    False
26     True
27     True
28    False
29    False
30    False
31    False
32     True
33    False
34    False
35     True
36    False
37    False
38    False
39    False
40    False
41    False
42     True
43    False
44    False
45     True
46    False
47    False
48     True
49    False
50    False
51    False
52    False
53     True
54    False
55    False
56    False
57    False
58    False
59    False
60    False
61    False
62    False
63    False
64    False
65    False
66    False
67    False
68    False
69    False
70    False
71    False
72    False
73    False
74    False
75    False
76    False
77     True
78    False
79    False
80     True
81     True
82    False
83  

In [10]:
import datetime

In [13]:
ts = df1.time.apply(lambda time: datetime.datetime.fromtimestamp(int(time)))

In [35]:
datetime.datetime.utcfromtimestamp(df1.time[1])

datetime.datetime(2015, 3, 25, 12, 59, 59)

In [21]:
t = datetime.datetime.fromtimestamp(df1.time[1])

In [37]:
from datetime import datetime

In [14]:
df1.time[1]

1427288399

In [28]:
datetime.datetime.fromtimestamp

<function datetime.fromtimestamp>

In [26]:
datetime.datetime.fromtimestamp(df1.ix[1, 'time'] , tz='utc')

TypeError: tzinfo argument must be None or of a tzinfo subclass, not type 'str'

In [17]:
df1.head()

,browser,user_hash,global_hash,url,time,country,repeat_client,keyword_alias,custom_keyword,city,timezone,label
0,Facebook,1H8wL5c,1H8wL5d,http://www.airasia.com/au/en/promotion/rr05509...,1427288399,AU,1,None,None,Perth,Australia/West,3
1,IE,1xupVE6,1lj67KQ,https://cdn.adf.ly/js/display.js,1427288399,US,0,None,None,Mountain View,America/Los_Angeles,5
2,IE,1xupVE6,1lj67KQ,https://cdn.adf.ly/js/display.js,1427288399,US,0,None,None,Buffalo,America/New_York,5
3,Facebook,1z7y10j,1z7y10k,https://predikta.com/Account/Register,1427288399,GB,1,None,None,Leicester,Europe/London,3
4,Mobile Safari,1pv5AIm,1pv5AIn,http://www.hasbro.com/en_US/brands/telepods/tr...,1427288399,CN,0,RTLPHT0814,None,Shanghai,Asia/Shanghai,7


In [20]:
df1.ix[1, 'timezone']

'America/Los_Angeles'

In [22]:
t

datetime.datetime(2015, 3, 25, 12, 59, 59)

In [38]:
t_utc = pd.Timestamp(t, tz='utc')
t_utc.hour

12

In [35]:
t_local = t_utc.tz_convert(df1.ix[1, 'timezone'])

In [36]:
t_local

Timestamp('2015-03-25 05:59:59-0700', tz='America/Los_Angeles')

In [37]:
t_local.hour

5

In [39]:
pd.Timestamp?